In [1]:
import os
from semantic_kernel.agents import AssistantAgentThread, AzureAssistantAgent
from azure.ai.agents.models import CodeInterpreterTool
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread
from azure.identity import EnvironmentCredential, get_bearer_token_provider
from dotenv import load_dotenv

load_dotenv()
pool_management_endpoint = os.getenv("POOL_MANAGEMENT_ENDPOINT")
ai_project_endpoint = os.getenv("AI_PROJECT_ENDPOINT")
chat_endpoint=os.getenv("OPENAI_ENDPOINT")
model_deployment_name=os.getenv("OPENAI_DEPLOYMENT_NAME")

## Semantic Kernel + AOAI Assistants API

In [15]:
# 1. Create the client using Azure OpenAI resources and configuration
provider = get_bearer_token_provider(EnvironmentCredential(), "https://cognitiveservices.azure.com/.default")
client = AzureAssistantAgent.create_client(
    endpoint=chat_endpoint, 
    ad_token_provider=provider, 
    deployment_name=model_deployment_name,
    api_version="2025-04-01-preview"
)

# 2. Configure the code interpreter tool and resources for the Assistant
code_interpreter_tool, code_interpreter_tool_resources = AzureAssistantAgent.configure_code_interpreter_tool()

# 3. Create the assistant on the Azure OpenAI service
definition = await client.beta.assistants.create(
    model=model_deployment_name,
    name="CodeRunner",
    instructions="Run the provided request as code and return the result. ALWAYS USE THE CODE INTERPRETER TOOL.",
    tools=code_interpreter_tool,
    tool_resources=code_interpreter_tool_resources,
)

# 4. Create a Semantic Kernel agent for the Azure OpenAI assistant
agent = AzureAssistantAgent(
    client=client,
    definition=definition,
)

# 5. Create a new thread for use with the assistant
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread: AssistantAgentThread = None

TASK = """Use the python code interpreter, 
print the current python version
then print the current date and time
then determine the values in the Fibonacci sequence that that are less than the value of 101?"""

print(f"# User: '{TASK}'")
try:
    # 6. Invoke the agent for the current thread and print the response
    async for response in agent.invoke(messages=TASK, thread=thread):
        print(f"# Agent: {response}")
        thread = response.thread
finally:
    # 7. Clean up the resources
    await thread.delete() if thread else None
    await agent.client.beta.assistants.delete(agent.id)


# User: 'Use the python code interpreter, 
print the current python version
then print the current date and time
then determine the values in the Fibonacci sequence that that are less than the value of 101?'
# Agent: # Importing necessary libraries
import sys
from datetime import datetime

# Current Python version
python_version = sys.version

# Current date and time
current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Generate Fibonacci sequence values less than 101
def fibonacci_sequence(limit):
    sequence = [0, 1]
    while sequence[-1] + sequence[-2] < limit:
        sequence.append(sequence[-1] + sequence[-2])
    return sequence

fibonacci_values_less_than_101 = fibonacci_sequence(101)

python_version, current_datetime, fibonacci_values_less_than_101
# Agent: - Current Python version: `3.11.12`
- Current date and time: `2025-06-20 15:29:42`
- Fibonacci sequence values less than 101: `[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]`


## Semantic Kernel + AI Foundry Agent SDK

Authentication is also handled with service principals - just need to ensure the SPN has a role assignment to the resource with the `Azure AI User` role.

In [16]:
# 1. Create client with connection string and auth to AI Foundry project
client = AzureAIAgent.create_client(endpoint=ai_project_endpoint, credential=EnvironmentCredential())

# 2. Create an agent with a code interpreter on the Azure AI agent service
code_interpreter = CodeInterpreterTool(file_ids=[])
agent_definition = await client.agents.create_agent(
    model=model_deployment_name,
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
    instructions="Run the provided request as code and return the result. ALWAYS USE THE CODE INTERPRETER TOOL.",
)

# 3. Create a Semantic Kernel agent for the Azure AI agent
agent = AzureAIAgent(
    client=client,
    definition=agent_definition,
)

# 4. Create a thread for the agent
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread: AzureAIAgentThread | None = None

TASK = """Use the python code interpreter, 
print the current python version
then print the current date and time
then determine the values in the Fibonacci sequence that that are less than the value of 101?"""
print(f"# User: '{TASK}'")

try:
    # 5. Invoke the agent for the specified thread for response
    async for response in agent.invoke(messages=TASK, thread=thread):
        print(f"# Agent: {response}")
        thread = response.thread
finally:
    # 6. Cleanup: Delete the thread, agent, and file
    await thread.delete() if thread else None
    await client.agents.delete_agent(agent.id)


# User: 'Use the python code interpreter, 
print the current python version
then print the current date and time
then determine the values in the Fibonacci sequence that that are less than the value of 101?'
# Agent: # Import necessary libraries
import sys
from datetime import datetime

# Print the current Python version
python_version = sys.version

# Get the current date and time
current_datetime = datetime.now()

# Generate the Fibonacci sequence less than 101
def fibonacci_less_than(n):
    sequence = [0, 1]
    while sequence[-1] + sequence[-2] < n:
        sequence.append(sequence[-1] + sequence[-2])
    return sequence

fibonacci_sequence = fibonacci_less_than(101)

python_version, current_datetime, fibonacci_sequence
# Agent: - The current Python version is `3.11.12 (main, May 22 2025, 01:28:11) [GCC 12.2.0]`.
- The current date and time is `2025-06-20 15:30:26`.
- The Fibonacci sequence values that are less than 101 are `[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]`.
